# Load training set and train paragraph vectors
Note: the paragraph vector model has been trained and is downloaded in the `prepare_feature_extraction()` function.

Retraining is therefore not needed, but optional

In [1]:
#%load_ext autoreload
#%autoreload 2

# If you need fully deterministic results between runs, set the following environment value prior to launching jupyter.
# See comment in sherlock.features.paragraph_vectors.infer_paragraph_embeddings_features for more info.
%env PYTHONHASHSEED =13

env: PYTHONHASHSEED=13


In [4]:
import multiprocessing as mp
import sys

from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pyarrow.parquet import ParquetFile

from sherlock import helpers
from sherlock.features.paragraph_vectors import (
    initialise_nltk,
    tagcol_paragraph_embeddings_features,
    train_paragraph_embeddings_features
)
from sherlock.features.preprocessing import convert_string_lists_to_lists
from sherlock.functional import extract_features_to_csv

print(f'Started at {datetime.now()}')

Started at 2022-09-13 16:04:49.493998


## Download and read in raw data


In [5]:
helpers.download_data()

Data was downloaded.


In [6]:
train_samples = pd.read_parquet('../data/data/raw/train_values.parquet')
train_labels = pd.read_parquet('../data/data/raw/train_labels.parquet')

In [7]:
print(train_samples.head(10))
print(train_labels.head(10))

                                                   values
55030            ['Global', 'United States', 'Australia']
167000  ['Fiction, Adult - Non-Floating', 'Fiction, Ad...
638282  ['', '', 'University of Puerto Rico - Rio Pied...
232298  ['Laughology', 'MTV', 'With Intent to Kill', '...
316158  ['Mare', 'Gelding', 'Gelding', 'Gelding', 'Gel...
467776  ['V.P., General Counsel & Sec.', 'V.P., Genera...
149640  ['GAJA', 'OREG', 'UCS', 'WCM', 'SLAM', 'ARIZ',...
23556   ['Applied Mathematics, University of Notre Dam...
263802  ['wakeup time in seconds for pbid to run its c...
476881     [35.0, 4.0, 52.0, 0.0, 30.0, 64.0, 84.0, None]
               type
55030          area
167000   collection
638282    team Name
232298       credit
316158       gender
467776     position
149640         club
23556   affiliation
263802  description
476881     position


In [8]:
train_samples_converted, y_train = convert_string_lists_to_lists(train_samples, train_labels, "values", "type")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 412059/412059 [00:37<00:00, 11036.45it/s]

types
<class 'pandas.core.series.Series'>
<class 'list'>


In [10]:
print(train_samples_converted.shape)

print(train_samples_converted.iloc[0], y_train[0])
print(train_samples_converted.iloc[412058], y_train[412058])


(412059,)
['Global', 'United States', 'Australia'] area
['Norwegian Cod Liver Oil Cherry', 'Norwegian Cod Liver Oil Mint'] product


In [11]:
#print(train_samples_converted.head)

print(train_samples_converted.head(10))
print(y_train[:10])

55030                    [Global, United States, Australia]
167000    [Fiction, Adult - Non-Floating, Fiction, Adult...
638282    [, , University of Puerto Rico - Rio Piedras, ...
232298    [Laughology, MTV, With Intent to Kill, Comedy ...
316158    [Mare, Gelding, Gelding, Gelding, Gelding, Mar...
467776    [V.P., General Counsel & Sec., V.P., General C...
149640    [GAJA, OREG, UCS, WCM, SLAM, ARIZ, NEM, VEN, M...
23556     [Applied Mathematics, University of Notre Dame...
263802    [wakeup time in seconds for pbid to run its ch...
476881           [35.0, 4.0, 52.0, 0.0, 30.0, 64.0, 84.0, ]
Name: values, dtype: object
['area', 'collection', 'team Name', 'credit', 'gender', 'position', 'club', 'affiliation', 'description', 'position']


## Train Doc2Vec

In [9]:
initialise_nltk()

[nltk_data] Downloading package punkt to /home/sunny/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Initialised NLTK, process took 0:00:00.473426 seconds.


[nltk_data] Downloading package stopwords to /home/sunny/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
samples = train_samples_converted.dropna()
print(f'Samples: {type(samples)}, length={len(samples)}')

train_labels = train_labels.dropna()

print(f'Labels:  {type(train_labels)}, length={len(train_labels)}')
#print(train_labels) #df

labels = train_labels.values.flatten()
print(f'Labels:  {type(labels)}, length={len(labels)}')

Samples: <class 'pandas.core.series.Series'>, length=412059
Labels:  <class 'pandas.core.frame.DataFrame'>, length=412059
Labels:  <class 'numpy.ndarray'>, length=412059


In [13]:
print(samples.head(10))
print(labels[:10])

samples = samples.head(10)
labels = labels[:10]

55030                    [Global, United States, Australia]
167000    [Fiction, Adult - Non-Floating, Fiction, Adult...
638282    [, , University of Puerto Rico - Rio Piedras, ...
232298    [Laughology, MTV, With Intent to Kill, Comedy ...
316158    [Mare, Gelding, Gelding, Gelding, Gelding, Mar...
467776    [V.P., General Counsel & Sec., V.P., General C...
149640    [GAJA, OREG, UCS, WCM, SLAM, ARIZ, NEM, VEN, M...
23556     [Applied Mathematics, University of Notre Dame...
263802    [wakeup time in seconds for pbid to run its ch...
476881           [35.0, 4.0, 52.0, 0.0, 30.0, 64.0, 84.0, ]
Name: values, dtype: object
['area' 'collection' 'team Name' 'credit' 'gender' 'position' 'club'
 'affiliation' 'description' 'position']


In [14]:
start = datetime.now()

print('Tagging columns')
cols = tagcol_paragraph_embeddings_features(samples, labels)

#print(cols)
print(f'Tagged Columns Doc2Vec Model, process took {datetime.now() - start} seconds.')

Tagging columns
Tagged Columns Doc2Vec Model, process took 0:00:00.011392 seconds.


In [15]:
#paragraph_vectors.py
import random; import nltk; from nltk.corpus import stopwords; from gensim.models.doc2vec import Doc2Vec, TaggedDocument
STOPWORDS_ENGLISH = stopwords.words("english")

def tokenise(values):
    joined = " ".join(s for s in values if len(s) >= 2)

    # stopwords need apostrophe
    filtered = "".join(
        e for e in joined if e.isalnum() or e.isspace() or e == "'"
    ).lower()

    return [
        word
        for word in nltk.word_tokenize(filtered)
        if len(word) >= 2 and word not in STOPWORDS_ENGLISH
    ]

def tagcol_paragraph_embeddings_features_nb(train_data: pd.Series, train_labels: list):
    random.seed(13)

    columns = []

    for i, col in enumerate(train_data):
        label = train_labels[i]
        values = random.sample(col, min(1000, len(col)))

        if len(values) > 0:
            values = list(map(lambda s: "" if s is None else str(s), values))

        tokens = tokenise(values)

        columns.append(TaggedDocument(tokens, label))

    return columns


In [16]:
start = datetime.now()

cols = tagcol_paragraph_embeddings_features_nb(samples, labels[:10])

print(cols[0])
print(cols[0].words)
print(cols[0].tags, '\n')

print(f'Tagged Columns Doc2Vec Model, process took {datetime.now() - start} seconds.', '\n')

print(cols)

TaggedDocument(['united', 'states', 'australia', 'global'], area)
['united', 'states', 'australia', 'global']
area 

Tagged Columns Doc2Vec Model, process took 0:00:00.003630 seconds. 

[TaggedDocument(words=['united', 'states', 'australia', 'global'], tags='area'), TaggedDocument(words=['fiction', 'adult', 'fiction', 'adult', 'fiction', 'adult', 'nonfloating', 'fiction', 'adult', 'fiction', 'adult'], tags='collection'), TaggedDocument(words=['university', 'puerto', 'rico', 'rio', 'piedras', 'parck', 'place', 'dealerships', 'sun', 'university', 'puerto', 'ricorio', 'piedras', 'university', 'puerto', 'ricorio', 'piedras', 'park', 'place', 'dealerships', 'university', 'puerto', 'rico', 'rio', 'piedras', 'park', 'place', 'dealerships', 'university', 'puerto', 'ricorio', 'piedras', 'parck', 'place', 'dealerships', 'sun', 'park', 'place', 'dealerships', 'park', 'place', 'dealerships', 'park', 'place', 'dealerships', 'park', 'place', 'dealerships', 'carolina', 'tri', 'university', 'puerto', 

In [17]:
start = datetime.now()

vec_dim = 400
print(f'Training Doc2Vec model in {vec_dim} dimensions')

train_paragraph_embeddings_features(cols, vec_dim)

print(f'Trained Doc2Vec Model, {vec_dim} dim, process took {datetime.now() - start} seconds.')

Training Doc2Vec model in 400 dimensions
Trained Doc2Vec Model, 400 dim, process took 0:00:00.101217 seconds.


In [28]:
print(f'Finished at {datetime.now()}')

Finished at 2022-03-29 16:59:23.854041


In [21]:
# need to save pkl + 3 npy files but missing one npy file

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing


# Train Doc2Vec model

train_model = Doc2Vec(
    cols,
    dm=0,
    negative=3,
    workers=multiprocessing.cpu_count(),
    vector_size=vec_dim,
    epochs=2,
    min_count=2,
    seed=13,
)

# Save trained model
#model_file = f"../sherlock/features/par_vec_trained_{vec_dim}.pkl"
model_file = f"../features/par_vec_trained_{vec_dim}.pkl"

train_model.save(model_file)
train_model.delete_temporary_training_data(
    keep_doctags_vectors=True, keep_inference=True
)

#print(train_model.docvecs.most_similar(0))



In [23]:
doc_words1 = ["last", "Deployment" ,"early" ,"other" ,"the work", "impact", "receive" ,"Behind the back" ,"Tsukuri", "trick" ,"Every time" ,"thing", "Take off your hat", "To do", "Read", "Cheap" ,"Me" ,"Mystery"]
doc_words2 = [ "Initiation love", "Similarly" ,"last", "A few lines", "Plot twist", "Go", "Time", "Time", "various", "scene", "To do" ,"To be", "Foreshadowing" ,"Sprinkle", "らTo be" ,"Is", "thing", "notice"]

doc_words3 = ["computer", "it", "science","python", "data", "database"]
doc_words4 = ["python", "data", "database","computer", "it", "science"]

# convert test/unseen paragraph to vector
print(train_model.infer_vector(doc_words1))


[-1.1604917e-03  2.5405362e-04 -4.7059624e-05 -5.8689527e-04
 -9.1203261e-04 -4.7114867e-04 -1.1892943e-04 -3.7765215e-04
 -6.9877534e-04 -9.8400656e-04  5.3689361e-04  1.1438956e-03
  6.7346054e-04 -7.3486770e-04  9.7964285e-04 -4.0206686e-04
  4.6434408e-04  8.1705773e-04  1.0198189e-03 -1.0774693e-04
  8.5484330e-04 -1.1690875e-03 -1.6510856e-04  2.6501023e-04
  7.9718063e-04 -7.8714511e-04  8.5567194e-04 -4.1868506e-04
 -3.7316902e-04 -7.7364163e-04 -1.1254011e-03 -9.4433711e-04
  6.2381872e-04 -2.0078970e-04  9.5900912e-05 -9.7853318e-04
  6.1046885e-04  8.6656638e-04 -6.1898585e-04  4.3883212e-04
 -3.7723588e-04 -2.1369127e-04 -1.1882943e-04 -1.6242257e-04
 -1.6104417e-04 -4.9740116e-05  4.3501714e-04  6.4576609e-04
 -9.9690782e-04 -1.8289508e-04  5.8654702e-04 -1.0299077e-03
  8.9984504e-04 -1.8582179e-04  3.7929762e-04  7.6381104e-05
 -5.5838079e-04 -7.5596955e-04  1.2036795e-03  1.0680333e-03
  8.8456413e-04  1.2139594e-03 -1.2422567e-03 -5.3944753e-04
 -8.0899731e-04 -1.00304

In [ ]:
print(train_model.docvecs.most_similar(0))



sim_value =train_model.docvecs.similarity_unseen_docs(train_model, doc_words1, doc_words4, alpha=1, min_alpha=0.0001, steps=5)
print(sim_value)

sim_value =train_model.docvecs.similarity_unseen_docs(train_model, doc_words3, doc_words4, alpha=1, min_alpha=0.0001, steps=5)
print(sim_value)

print(cols[0])


print(train_model)
print(train_model.docvecs.most_similar(0))


print(model)
print(model.docvecs.most_similar('0'))


print(train_model.most_similar(positive=['woman', 'king'], negative=['man']))
print(train_model.most_similar(positive=['country']))

In [91]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]



max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

    
#model_file = f"../sherlock/features/par_vec_trained_{vec_dim}.pkl"
model.save("d2v.model")
print("Model Saved")


/home/sunny/miniconda3/envs/env5/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40


/home/sunny/miniconda3/envs/env5/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteration 78
iteration 79
iteration 80
iteration 81
iteration 82
iteration 83
iteration 84
iteration 85
iteration 86
iteration 87
iteration 88
iteration 89
iteration 90
iteration 91
iteration 92
iteration 93
iteration 94
iteration 95
iteration 96
iteration 97
iteration 98
iteration 99
Model Saved
V1_infer [ 0.0165429  -0.00657688 -0.02399274 -0.01181185  0.00387255 -0.02063944
 -0.01466336 -0.00024349  0.01962049 -0.02046933 -0.03034732  0.0260456
 -0.0088838  -0.01737959 -0.01878665 -0.00285814  0.02368273 -0.013

TypeError: '<' not supported between instances of 'str' and 'int'

In [92]:


from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")


#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])

V1_infer [ 0.0165429  -0.00657688 -0.02399274 -0.01181185  0.00387255 -0.02063944
 -0.01466336 -0.00024349  0.01962049 -0.02046933 -0.03034732  0.0260456
 -0.0088838  -0.01737959 -0.01878665 -0.00285814  0.02368273 -0.01362796
 -0.00856395 -0.0003635 ]
[('0', 0.9920483231544495), ('2', 0.9915789365768433), ('3', 0.9913952350616455)]
[-0.20910682  0.27863237 -0.21247207  0.00621525  0.44725454  0.10419783
 -0.12027526 -0.07403916 -0.15418532  0.1021312  -0.2671689   0.47951084
  0.2586121   0.12724744  0.25313672  0.09417614  0.6556045  -0.13270836
  0.09749547  0.1739856 ]
